In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install grad-cam

In [ ]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image, ImageChops
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset

# This is for the progress bar.
from tqdm.auto import tqdm
import random

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import cv2
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

In [ ]:
dataset_path = "../input/chest-xray-pneumonia/chest_xray"
pneumonia_dir = dataset_path+"/PNEUMONIA"
normal_dir = dataset_path+"/NORMAL"
pneumonia_files = sorted(os.listdir(os.path.join(dataset_path, 'PNEUMONIA')))
normal_files = sorted(os.listdir(os.path.join(dataset_path, 'NORMAL')))

In [ ]:
print(len(pneumonia_files), len(normal_files))

In [ ]:
width = []
height = []
for i in range(len(pneumonia_files)):
    img = plt.imread(os.path.join(pneumonia_dir, pneumonia_files[i]))
    height.append(img.shape[0])
    width.append(img.shape[1])
    # break
    
plt.title("Image Size Distribution of PNEUMONIA")
plt.xlabel("Width")
plt.ylabel("Height")
plt.scatter(width, height)

print(np.polynomial.polynomial.polyfit(width, height, 1))


In [ ]:
width = []
height = []
for i in range(len(normal_files)):
    img = plt.imread(os.path.join(normal_dir, normal_files[i]))
    height.append(img.shape[0])
    width.append(img.shape[1])
    
plt.title("Image Size Distribution of NORMAL")
plt.xlabel("Width")
plt.ylabel("Height")
plt.scatter(width, height)
print(np.polynomial.polynomial.polyfit(width, height, 1))

In [ ]:
plt.figure(figsize=(16, 6))
plt.suptitle('Raw PNEUMONIA')

for i in range(8):
    plt.subplot(2, 4, i + 1)
    img = plt.imread(os.path.join(pneumonia_dir, pneumonia_files[i]))
    plt.imshow(img, cmap='gray')
    # plt.axis('off')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(16, 6))
plt.suptitle('Raw NORMAL')

for i in range(8):
    plt.subplot(2, 4, i + 1)
    img = plt.imread(os.path.join(normal_dir, normal_files[i]))
    plt.imshow(img, cmap='gray')
    # plt.axis('off')
plt.tight_layout()

## Transformation

In [ ]:

test_tfm = transforms.Compose([
    transforms.Resize((299, 299), antialias =True),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),

])

train_tfm = transforms.Compose([
    transforms.Resize((299, 299), antialias =True),
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomAffine(degrees=20, translate=(0.15,0.2)),
    # transforms.RandomRotation(30),
    transforms.ToTensor(),
])

In [ ]:
plt.figure(figsize=(14, 7))
plt.suptitle('Resized PNEUMONIA')
 
for i in range(8):
    img = Image.open(os.path.join(pneumonia_dir, pneumonia_files[i]))
    img = test_tfm(img)
    plt.subplot(2, 4, i + 1)
    plt.imshow(img.permute(1, 2, 0))
    # plt.axis('off')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(14, 7))
plt.suptitle('Resized NORMAL')
 
for i in range(8):
    img = Image.open(os.path.join(normal_dir, normal_files[i]))
    img = test_tfm(img)
    plt.subplot(2, 4, i + 1)
    plt.imshow(img.permute(1, 2, 0))
    # plt.axis('off')
plt.tight_layout()

In [ ]:
img = plt.imread(os.path.join(pneumonia_dir, pneumonia_files[0]))
img

In [ ]:
img = Image.open(os.path.join(pneumonia_dir, pneumonia_files[0]))
tfm_img = train_tfm(img)
tfm_img

In [ ]:
total = np.zeros((256,256, 3))
# print(total)
for i in range(len(pneumonia_files)):
    img = Image.open(os.path.join(pneumonia_dir, pneumonia_files[i]))
    img = np.array(test_tfm(img).permute(1,2,0))
    # print(img)
    total = total + img

p_mean = total / len(pneumonia_files)
plt.imshow(p_mean)
plt.title("Mean PNEUMONIA")

In [ ]:
total = np.zeros((256,256, 3))
# print(total)
for i in range(len(normal_files)):
    img = Image.open(os.path.join(normal_dir, normal_files[i]))
    img = np.array(test_tfm(img).permute(1,2,0))
    # print(img)
    total = total + img

n_mean = total / len(normal_files)
plt.imshow(n_mean)
plt.title("Mean NORMAL")

In [ ]:
diff = p_mean - n_mean
diff = diff[:, :, 0]
ax = sns.heatmap(diff, cmap="coolwarm")
ax.set_title('Mean PNEUMONIA - Mean NORMAL')
plt.show()

In [ ]:
square_sum = np.zeros((256,256, 3))
for i in range(len(pneumonia_files)):
    img = Image.open(os.path.join(pneumonia_dir, pneumonia_files[i]))
    img = np.array(test_tfm(img).permute(1,2,0))
    square_sum = square_sum + np.power(img - p_mean, 2)
    
p_var = square_sum /  len(pneumonia_files)
p_var = p_var[:, :, 0]
p_var_max = np.max(p_var)
ax = sns.heatmap(p_var, cmap="coolwarm", vmax=0.125)
ax.set_title("Variance PNEUMONIA [Max = {:.3f}]".format(p_var_max))
plt.show()

In [ ]:
square_sum = np.zeros((256,256, 3))
for i in range(len(normal_files)):
    img = Image.open(os.path.join(normal_dir, normal_files[i]))
    img = np.array(test_tfm(img).permute(1,2,0))
    square_sum = square_sum + np.power(img - n_mean, 2)

n_var = square_sum /  len(normal_files)
n_var = n_var[:, :, 0]
n_var_max = np.max(n_var)
ax = sns.heatmap(n_var, cmap="coolwarm", vmax=0.125)
ax.set_title("Variance NORMAL [Max = {:.3f}]".format(n_var_max))
plt.show()

In [ ]:
def train_valid_test_split(folder, train_size, valid_size, test_size):
    # 0 = train, 1 = valid, 2 = test
    
    train_files=[]
    valid_files=[]
    test_files=[]
    
    for file in folder:
        if len(train_files) <= train_size:
            train_files.append(file)
        elif len(valid_files) <= (valid_size):
            valid_files.append(file)
        else:
            test_files.append(file)
  
    return train_files, valid_files, test_files


In [ ]:
p_train_files, p_valid_files, p_test_files = train_valid_test_split(pneumonia_files, 3673, 300, 300)
n_train_files, n_valid_files, n_test_files = train_valid_test_split(normal_files, 1083, 250, 250)

In [ ]:
class ChestXRayDataset(Dataset):

    def __init__(self,files, label, tfm=None):
        self.files = files
        self.transform = tfm
        self.label = label
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        
        return im, self.label

In [ ]:
n_train_set = ChestXRayDataset([os.path.join(normal_dir, x) for x in n_train_files], label=0, tfm=train_tfm)
n_valid_set = ChestXRayDataset([os.path.join(normal_dir, x) for x in n_valid_files], label=0, tfm=test_tfm)
n_test_set = ChestXRayDataset([os.path.join(normal_dir, x) for x in n_test_files], label=0, tfm=test_tfm)

p_train_set = ChestXRayDataset([os.path.join(pneumonia_dir, x) for x in p_train_files], label=1, tfm=train_tfm)
p_valid_set = ChestXRayDataset([os.path.join(pneumonia_dir, x) for x in p_valid_files], label=1, tfm=test_tfm)
p_test_set = ChestXRayDataset([os.path.join(pneumonia_dir, x) for x in p_test_files], label=1, tfm=test_tfm)

In [ ]:
train_set = n_train_set + p_train_set
valid_set = n_valid_set + p_valid_set
test_set = n_test_set + p_test_set

In [ ]:
print(len(train_set), len(valid_set), len(test_set))

In [ ]:
batch_size = 16

In [ ]:
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

In [ ]:
import torchvision.models as models
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        num_classes = 128
        self.model1 = models.densenet121(pretrained=True)
        self.model1.classifier = nn.Linear(1024, num_classes)
        
        self.model2 = models.resnet18(pretrained=True)
        self.model2.fc = nn.Linear(512, num_classes)
        
        self.model3 = models.inception_v3(pretrained=True, aux_logits=False)
        # self.model3.AuxLogits.fc = nn.Linear(768, num_classes)
        self.model3.fc = nn.Linear(2048, num_classes)
        
        self.fc = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2),
            
        )

    def forward(self, x):
        out1 = self.model1(x)                 # size: [batch, 2]
        
        out2 = self.model2(x)                 # size: [batch, 2]
        
        out3 = self.model3(x)
        
        # out = torch.cat((out1, out2, out3), dim=-1)
        out = out1 + out2 + out3
        
        return self.fc(out)

In [ ]:
_exp_name = "dlmi-mid"
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
n_epochs =30
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss()

# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=1e-4) 
optimizer = torch.optim.Adam(model.parameters())

best_acc = 0
all_train_loss = []
all_train_accs = []
all_valid_loss = []
all_valid_accs = []

for epoch in range(n_epochs):

    model.train()

    train_loss = []
    train_accs = []

    for batch in tqdm(train_loader):

        # ---------- Training ----------

        imgs, labels = batch

        logits = model(imgs.to(device))

        loss = criterion(logits, labels.to(device))

        optimizer.zero_grad()

        loss.backward()

        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        optimizer.step()

        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        train_loss.append(loss.item())
        train_accs.append(acc)

    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)
    
    all_train_loss.append(train_loss)
    all_train_accs.append(train_acc)
    
    
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    model.eval()

    valid_loss = []
    valid_accs = []

    for batch in tqdm(valid_loader):

        imgs, labels = batch
        with torch.no_grad():
            logits = model(imgs.to(device))

        loss = criterion(logits, labels.to(device))

        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        valid_loss.append(loss.item())
        valid_accs.append(acc)

    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)
    
    all_valid_loss.append(valid_loss)
    all_valid_accs.append(valid_acc)
    
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # save models
    if valid_acc > best_acc:
        print(f"Best model found at epoch {epoch}, saving model")
        torch.save(model.state_dict(), f"{_exp_name}_best.ckpt") # only save best to prevent output memory exceed error
        best_acc = valid_acc
        stale = 0

In [ ]:
plt.plot(range(n_epochs), all_train_loss, label="Train")
plt.plot(range(n_epochs), all_valid_loss, label="Valid")
plt.legend(loc="upper right")
plt.xlabel("Epoch")
plt.ylabel("Loss")

In [ ]:
all_train_accs_cpu = [i.cpu().numpy() for i in all_train_accs]
all_valid_accs_cpu = [i.cpu().numpy() for i in all_valid_accs]
plt.plot(range(n_epochs), all_train_accs_cpu, label="Train")
plt.plot(range(n_epochs), all_valid_accs_cpu, label="Valid")
plt.legend(loc="lower right")
plt.xlabel("Epoch")
plt.ylabel("Acc")

In [ ]:
target_layer1 = [model.model1.features[-1]]
target_layer1

In [ ]:
cam = GradCAM(model=model, target_layers=target_layer1, use_cuda=True)

In [ ]:
plt.figure(figsize=(12, 3))
plt.suptitle('DenseNet Grad-CAM, PNEUMONIA')
 
for i in range(4):
    plt.subplot(1, 4, i + 1)
    tensor_img = p_valid_set[i][0]
    input_tensor = tensor_img.unsqueeze(0) # [batch, channel, height, width]
    # If None, returns the map for the highest scoring category.
    # Otherwise, targets the requested category.
    targets = None
    # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
    grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

    # In this example grayscale_cam has only one image in the batch:
    grayscale_cam = grayscale_cam[0, :]
    img = np.array(tensor_img.permute(1,2,0))
    visualization = show_cam_on_image(img, grayscale_cam)
    
    plt.imshow(visualization)
    # plt.show()
    # plt.axis('off')
    
plt.tight_layout()

In [ ]:
plt.figure(figsize=(12, 3))
plt.suptitle('DenseNet Grad-CAM, NORMAL')
 
for i in range(4):
    plt.subplot(1, 4, i + 1)
    tensor_img = n_valid_set[i][0]
    input_tensor = tensor_img.unsqueeze(0) # [batch, channel, height, width]
    # If None, returns the map for the highest scoring category.
    # Otherwise, targets the requested category.
    targets = None
    # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
    grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

    # In this example grayscale_cam has only one image in the batch:
    grayscale_cam = grayscale_cam[0, :]
    img = np.array(tensor_img.permute(1,2,0))
    visualization = show_cam_on_image(img, grayscale_cam)
    
    plt.imshow(visualization)
    # plt.show()
    # plt.axis('off')
    
plt.tight_layout()

In [ ]:
target_layer2 = [model.model2.layer4[-1]]
print(target_layer2)
cam = GradCAM(model=model, target_layers=target_layer2, use_cuda=True)

In [ ]:
plt.figure(figsize=(12, 3))
plt.suptitle('ResNet Grad-CAM, PNEUMONIA')
 
for i in range(4):
    plt.subplot(1, 4, i + 1)
    tensor_img = p_valid_set[i][0]
    input_tensor = tensor_img.unsqueeze(0) # [batch, channel, height, width]
    # If None, returns the map for the highest scoring category.
    # Otherwise, targets the requested category.
    targets = None
    # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
    grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

    # In this example grayscale_cam has only one image in the batch:
    grayscale_cam = grayscale_cam[0, :]
    img = np.array(tensor_img.permute(1,2,0))
    visualization = show_cam_on_image(img, grayscale_cam)
    
    plt.imshow(visualization)
    # plt.show()
    # plt.axis('off')
    
plt.tight_layout()

In [ ]:
plt.figure(figsize=(12, 3))
plt.suptitle('ResNet Grad-CAM, NORMAL')
 
for i in range(4):
    plt.subplot(1, 4, i + 1)
    tensor_img = n_valid_set[i][0]
    input_tensor = tensor_img.unsqueeze(0) # [batch, channel, height, width]
    # If None, returns the map for the highest scoring category.
    # Otherwise, targets the requested category.
    targets = None
    # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
    grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

    # In this example grayscale_cam has only one image in the batch:
    grayscale_cam = grayscale_cam[0, :]
    img = np.array(tensor_img.permute(1,2,0))
    visualization = show_cam_on_image(img, grayscale_cam)
    
    plt.imshow(visualization)
    # plt.show()
    # plt.axis('off')
    
plt.tight_layout()

In [ ]:
target_layer3 = [model.model3.Mixed_7c]
print(target_layer3)
cam = GradCAM(model=model, target_layers=target_layer3, use_cuda=True)

In [ ]:
plt.figure(figsize=(12, 3))
plt.suptitle('Inception V3 Grad-CAM, PNEUMONIA')
 
for i in range(4):
    plt.subplot(1, 4, i + 1)
    tensor_img = p_valid_set[i][0]
    input_tensor = tensor_img.unsqueeze(0) # [batch, channel, height, width]
    # If None, returns the map for the highest scoring category.
    # Otherwise, targets the requested category.
    targets = None
    # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
    grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

    # In this example grayscale_cam has only one image in the batch:
    grayscale_cam = grayscale_cam[0, :]
    img = np.array(tensor_img.permute(1,2,0))
    visualization = show_cam_on_image(img, grayscale_cam)
    
    plt.imshow(visualization)
    # plt.show()
    # plt.axis('off')
    
plt.tight_layout()

In [ ]:
plt.figure(figsize=(12, 3))
plt.suptitle('Inception V3 Grad-CAM, NORMAL')

for i in range(4):
    plt.subplot(1, 4, i + 1)
    tensor_img = n_valid_set[i][0]
    input_tensor = tensor_img.unsqueeze(0) # [batch, channel, height, width]
    # If None, returns the map for the highest scoring category.
    # Otherwise, targets the requested category.
    targets = None
    # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
    grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

    # In this example grayscale_cam has only one image in the batch:
    grayscale_cam = grayscale_cam[0, :]
    img = np.array(tensor_img.permute(1,2,0))
    visualization = show_cam_on_image(img, grayscale_cam)
    
    plt.imshow(visualization)
    # plt.show()
    # plt.axis('off')
    
plt.tight_layout()

In [ ]:
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=0, pin_memory=True)

In [ ]:
model_best = Classifier().to(device)
model_best.load_state_dict(torch.load(f"{_exp_name}_best.ckpt"))
model_best.eval()
confusion_mat = np.zeros((2, 2))
cnt = 0

for batch in tqdm(test_loader):

    data, labels = batch

    with torch.no_grad():
        test_pred = model_best(data.to(device))
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)

    if (test_label == 1):
        if(test_label == np.array(labels)):
            confusion_mat[0][0] = confusion_mat[0][0] + 1
        else:
            confusion_mat[0][1] = confusion_mat[0][1] + 1
    elif (test_label == 0): 
        if(test_label == np.array(labels)):
            confusion_mat[1][1] = confusion_mat[1][1] + 1
        else:
            confusion_mat[1][0] = confusion_mat[1][0] + 1
        
total = confusion_mat[0][0] +  confusion_mat[1][0] +  confusion_mat[0][1] + confusion_mat[1][1]
print("confusion_mat", confusion_mat)
print("accuracy", (confusion_mat[0][0] + confusion_mat[1][1]) / total)
precision =  confusion_mat[0][0] / (confusion_mat[0][0] + confusion_mat[0][1])
recall = confusion_mat[0][0] / (confusion_mat[0][0] + confusion_mat[1][0])
print("precision", precision)
print("recall", recall)
print("f1", 2*precision*recall / (precision+recall))
